In [9]:
import pandas as pd

# Load dataset
df = pd.read_csv("dataset.csv")

# Only these features
features = ["tempo", "loudness", "danceability"]

# Quartiles
quartiles = df[features].quantile([0.25, 0.5, 0.75])
print(quartiles)


          tempo  loudness  danceability
0.25   99.21875   -10.013         0.456
0.50  122.01700    -7.004         0.580
0.75  140.07100    -5.003         0.695


In [14]:
def mood_mapper(tempo, loudness, danceability, q):
    # Loudness category
    if loudness >= q["loudness"].loc[0.75]:
        loud_cat = "high"
    elif loudness <= q["loudness"].loc[0.25]:
        loud_cat = "low"
    else:
        loud_cat = "mid"
    
    # Tempo category
    if tempo >= q["tempo"].loc[0.75]:
        tempo_cat = "fast"
    elif tempo <= q["tempo"].loc[0.25]:
        tempo_cat = "slow"
    else:
        tempo_cat = "mid"
    
    # Danceability category
    if danceability >= q["danceability"].loc[0.75]:
        dance_cat = "high"
    elif danceability <= q["danceability"].loc[0.25]:
        dance_cat = "low"
    else:
        dance_cat = "mid"

    # Mood rules (relaxed, not over-constrained)
    if tempo_cat == "fast" and dance_cat == "high" and loud_cat == "high":
        return "Happy / Energetic (High Beat Party)"
    
    elif tempo_cat in ["mid", "fast"] and dance_cat in ["mid", "high"] and loud_cat in ["mid", "high"]:
        return "Happy / Energetic (Pleasant)"
    
    elif tempo_cat == "mid" and dance_cat == "mid":
        return "Uplifting / Positive"
    
    elif tempo_cat == "slow" and loud_cat == "low":
        return "Calm / Peaceful"
    
    elif tempo_cat == "slow" and dance_cat == "low":
        return "Relaxation / Safety"
    
    elif tempo_cat == "mid" and dance_cat in ["mid", "low"]:
        return "Balanced / Neutral"
    
    else:
        # Instead of dumping, force it into closest vibe
        if tempo_cat == "fast":
            return "High Energy / Excited"
        elif tempo_cat == "slow":
            return "Calm / Peaceful"
        else:
            return "Balanced / Neutral"


In [15]:
df["MoodCategory"] = df.apply(
    lambda row: mood_mapper(row["tempo"], row["loudness"], row["danceability"], quartiles),
    axis=1
)

# Save
df.to_csv("dataset_with_moods_tld.csv", index=False)

# Distribution
print(df["MoodCategory"].value_counts())


MoodCategory
Happy / Energetic (Pleasant)           51442
Calm / Peaceful                        24577
Balanced / Neutral                     13245
High Energy / Excited                  13221
Uplifting / Positive                    6436
Relaxation / Safety                     3923
Happy / Energetic (High Beat Party)     1156
Name: count, dtype: int64
